# Get the departments and rooms within each departments

In [12]:
%matplotlib inline
import numpy as np
import compas
from compas.datastructures import Mesh
import compas.datastructures as ds
from compas_plotters import Plotter
import matplotlib.pyplot as plt
import networkx as nx 
import json
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [13]:
fin_real = open("C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/GENERATING_GML/output.json")
j_real = json.loads(fin_real.read())

## Get room names and areas

In [14]:
def get_names_and_areas(jsonfile):
    cellspaces = []
    room_names = []
    room_areas = []

    for i in jsonfile['PrimalSpaceFeatures']['CellSpace']:
        cellspaces.append(i)
    for cs in cellspaces:
        name = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['name']
        room_names.append(name)

    for cs in cellspaces:
        area = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['attributes']['area']
        room_areas.append(area)
    name_and_area = list(zip(room_names,room_areas))
    return name_and_area

roomName_and_area = get_names_and_areas(j_real)
print(roomName_and_area)
print(len(roomName_and_area))
for i in roomName_and_area:
    print(i)

# change this data structure into a dictionary, where the key is the room name, and the values are area, and others. the values can be tuple (such as crowdedness, etc.)

[('CENTRALWAITING1AC1', '127.0993141'), ('WAITING/ACTIVITYAREA1DC1', '178.2169708'), ('ROOF2R02', '578.7414997'), ('CORRIDOR1EC1', '50.950835'), ('ROOF3R01', '1729.774807'), ('MAINMECHANICALROOM2D05', '226.04322'), ('CORRIDOR2AC3', '25.4807999'), ('CORRIDOR1BC2', '27.31731525'), ('TECH.OFFICE2D18', '82.089156'), ('CORRIDOR1CC1', '40.23448'), ('PHARM.DISP.1A16', '65.22472'), ('CORRIDOR1DC3', '29.65476604'), ('PHYSICALEXAM1C18', '55.37747'), ('LAB1B04', '51.1950656'), ('CORRIDOR1BC4', '41.72148526'), ('CORRIDOR2BC3', '24.92304611'), ('CORRIDOR2BC5', '20.941674'), ('RECEIVING/STORAGE1E24', '115.1199912'), ('CORRIDOR2AC1', '39.02683601'), ('CORRIDOR1CC3', '10.608154'), ('PROVIDERCUBICLES1D16', '65.26142'), ('CORRIDOR1DC7', '29.82494475'), ('CORRIDOR2BC2', '27.59131613'), ('BMETWORKSTATION1E16', '46.7310228'), ('DENTALWAITING2A11', '31.7576639'), ('CORRIDOR2CC3', '16.08426'), ('CORRIDOR1AC5', '19.219196'), ('CORRIDOR2DC2', '28.37771404'), ('CORRIDOR1AC3', '15.00834681'), ('PROVIDERCUBICLES1

In [15]:
#make dict

def get_names_and_areas_dict(jsonfile):
    cellspaces = []
    room_names = []
    room_areas = []

    for i in jsonfile['PrimalSpaceFeatures']['CellSpace']:
        cellspaces.append(i)
    for cs in cellspaces:
        name = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['name']
        room_names.append(name)

    for cs in cellspaces:
        area = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['attributes']['area']
        room_areas.append({'area':area})
    #name_and_area = list(zip(room_names,room_areas))
    name_and_area = dict(map(lambda i,j : (i,j) , room_names,room_areas))
    return name_and_area

dict_room_Name_and_area = get_names_and_areas_dict(j_real)
print(dict_room_Name_and_area)
print(len(dict_room_Name_and_area))


{'CENTRALWAITING1AC1': {'area': '127.0993141'}, 'WAITING/ACTIVITYAREA1DC1': {'area': '178.2169708'}, 'ROOF2R02': {'area': '578.7414997'}, 'CORRIDOR1EC1': {'area': '50.950835'}, 'ROOF3R01': {'area': '1729.774807'}, 'MAINMECHANICALROOM2D05': {'area': '226.04322'}, 'CORRIDOR2AC3': {'area': '25.4807999'}, 'CORRIDOR1BC2': {'area': '27.31731525'}, 'TECH.OFFICE2D18': {'area': '82.089156'}, 'CORRIDOR1CC1': {'area': '40.23448'}, 'PHARM.DISP.1A16': {'area': '65.22472'}, 'CORRIDOR1DC3': {'area': '29.65476604'}, 'PHYSICALEXAM1C18': {'area': '55.37747'}, 'LAB1B04': {'area': '51.1950656'}, 'CORRIDOR1BC4': {'area': '41.72148526'}, 'CORRIDOR2BC3': {'area': '24.92304611'}, 'CORRIDOR2BC5': {'area': '20.941674'}, 'RECEIVING/STORAGE1E24': {'area': '115.1199912'}, 'CORRIDOR2AC1': {'area': '39.02683601'}, 'CORRIDOR1CC3': {'area': '10.608154'}, 'PROVIDERCUBICLES1D16': {'area': '65.26142'}, 'CORRIDOR1DC7': {'area': '29.82494475'}, 'CORRIDOR2BC2': {'area': '27.59131613'}, 'BMETWORKSTATION1E16': {'area': '46.73

## Organize rooms to their deparments

In [16]:

# Provided list of room data
data = [
    ('CENTRALWAITING1AC1', '127.0993141'),
    ('WAITING/ACTIVITYAREA1DC1', '178.2169708'),
    ('ROOF2R02', '578.7414997'),
    ('CORRIDOR1EC1', '50.950835'),
    ('CORRIDOR2AC3', '25.4807999')
]

# Create a dictionary to organize rooms by department
'''output_data = {}

# Loop through each item in the list to organize by department
for room, area in roomName_and_area:
    # Extract department character (the last third character of the room name)
    department = 'Department$' + room[-3]
    
    # Check if the department key exists in the dictionary
    if department not in output_data:
        # If not, create a new list for this department
        output_data[department] = []
        
    # Append the room and area tuple to the appropriate department list
    output_data[department].append((room, area))

output_data'''
def get_department_of_rooms(room_name_area):

    output_data = {}

    # Loop through each item in the list to organize by department
    for room, area in room_name_area:
        # Extract department character (the last third character of the room name)
        department = 'Department$' + room[-3]
        
        # Check if the department key exists in the dictionary
        if department not in output_data:
            # If not, create a new list for this department
            output_data[department] = []
            
        # Append the room and area tuple to the appropriate department list
        output_data[department].append((room, area))

    return output_data

output_data = get_department_of_rooms(roomName_and_area)
output_data


{'Department$A': [('CENTRALWAITING1AC1', '127.0993141'),
  ('CORRIDOR2AC3', '25.4807999'),
  ('PHARM.DISP.1A16', '65.22472'),
  ('CORRIDOR2AC1', '39.02683601'),
  ('DENTALWAITING2A11', '31.7576639'),
  ('CORRIDOR1AC5', '19.219196'),
  ('CORRIDOR1AC3', '15.00834681'),
  ('CORRIDOR2AC4', '13.3551'),
  ('RMOANALYST1A02', '26.65783112'),
  ('CORRIDOR1AC2', '19.09846713'),
  ('CORRIDOR2AC2', '13.92585484'),
  ('LIBRARY/CONF.ROOM2A01', '24.92560559'),
  ('ADMIN2A14', '31.49841497'),
  ('LOUNGE1A10', '9.519166011'),
  ('DEVELOPING2A13', '6.885029095'),
  ('X-RAY2A12', '10.34281402'),
  ('CORRIDOR1AC4', '8.799656433'),
  ('OHDTR#12A05', '10.44778121'),
  ('TRICAREOFFICE1A07', '7.969378'),
  ('COUNSELING1A15', '5.868032'),
  ('M.TOILET1A12', '9.146935978'),
  ('W.TOILET1A09', '9.817789575'),
  ('SUPEROFFICE2A02', '10.023352'),
  ('GEN.DENT.DTR#52A09', '10.39306421'),
  ('GEN.DENT.DTR#42A08', '10.7436992'),
  ('OHDTR#22A06', '11.00429787'),
  ('PATIENTADMIN.RECEPT.1A01', '19.016928'),
  ('TRICAR

In [17]:
test_dict = {
    'Department$A': [('CENTRALWAITING1AC1', '127.0993141'), ('CORRIDOR2AC3', '25.4807999')],
    'Department$D': [('WAITING/ACTIVITYAREA1DC1', '178.2169708'), ('MAINMECHANICALROOM2D05', '226.04322')],
    'Department$B': [('CORRIDOR1BC2', '27.31731525'), ('LAB1B04', '51.1950656')],
    'Department$E': [('CORRIDOR1EC1', '50.950835'), ('RECEIVING/STORAGE1E24', '115.1199912')],
    'Department$1': [('OPENTOBELOW2AC1-B', '74.36477597')],
    'Department$0': [('COMPUTERROOM2D04A', '32.0805114')],
    'Department$2': [
        ('X-RAYALCOVE2A12-A', '1.995578054'), ('OFFICE/EXAM2B8', '11.735796'), ('W.TOILET2B5', '4.130912'),
        ('TECHOFFICE2B7', '9.236736'), ('STAFFTOILET2B6', '5.012672'), ('JAN.CL.2B2', '1.8928'),
        ('TECHOFFICE2B9', '9.169216')
    ],
    'Department$F': [('MECH.YARD1F01', '78.91683985')],
    'Department$5': [('GASSTORAGE1E15-A', '1.6575762')]
}

# Function to find the correct department based on the room name
def find_correct_department(room_name):
    for char in room_name:
        if char.isdigit():
            # Find the first character after the digit and use it to determine the correct department
            index = room_name.index(char) + 1
            return 'Department$' + room_name[index].upper()  # Assuming department is determined by this logic
    return None

# Iterate over the keys and move the rooms to the correct department
'''keys_to_remove = ['Department$0', 'Department$1', 'Department$2', 'Department$5']
for key in keys_to_remove:
    rooms = output_data.pop(key, [])  # Remove the entry and get the rooms list
    for room in rooms:
        correct_department = find_correct_department(room[0])
        if correct_department:
            if correct_department in output_data:
                output_data[correct_department].append(room)
            else:
                output_data[correct_department] = [room]

output_data'''
def correct_department_dict(keys_to_remove,inital_dict):

    for key in keys_to_remove:
        rooms = inital_dict.pop(key, [])  # Remove the entry and get the rooms list
        for room in rooms:
            correct_department = find_correct_department(room[0])
            if correct_department:
                if correct_department in inital_dict:
                    inital_dict[correct_department].append(room)
                else:
                    inital_dict[correct_department] = [room]

    return inital_dict

keys_to_remove = ['Department$0', 'Department$1', 'Department$2', 'Department$5']
corrected_department = correct_department_dict(keys_to_remove,output_data)
corrected_department

{'Department$A': [('CENTRALWAITING1AC1', '127.0993141'),
  ('CORRIDOR2AC3', '25.4807999'),
  ('PHARM.DISP.1A16', '65.22472'),
  ('CORRIDOR2AC1', '39.02683601'),
  ('DENTALWAITING2A11', '31.7576639'),
  ('CORRIDOR1AC5', '19.219196'),
  ('CORRIDOR1AC3', '15.00834681'),
  ('CORRIDOR2AC4', '13.3551'),
  ('RMOANALYST1A02', '26.65783112'),
  ('CORRIDOR1AC2', '19.09846713'),
  ('CORRIDOR2AC2', '13.92585484'),
  ('LIBRARY/CONF.ROOM2A01', '24.92560559'),
  ('ADMIN2A14', '31.49841497'),
  ('LOUNGE1A10', '9.519166011'),
  ('DEVELOPING2A13', '6.885029095'),
  ('X-RAY2A12', '10.34281402'),
  ('CORRIDOR1AC4', '8.799656433'),
  ('OHDTR#12A05', '10.44778121'),
  ('TRICAREOFFICE1A07', '7.969378'),
  ('COUNSELING1A15', '5.868032'),
  ('M.TOILET1A12', '9.146935978'),
  ('W.TOILET1A09', '9.817789575'),
  ('SUPEROFFICE2A02', '10.023352'),
  ('GEN.DENT.DTR#52A09', '10.39306421'),
  ('GEN.DENT.DTR#42A08', '10.7436992'),
  ('OHDTR#22A06', '11.00429787'),
  ('PATIENTADMIN.RECEPT.1A01', '19.016928'),
  ('TRICAR